# Solução do problema do Caixeiro Viajante (TSP)

### Uso de quatro técnicas diferentes
1. Busca exaustiva
2. Greedy
3. Hill Climbing
4. Simulated Annealing

In [1]:
# Stephen Marsland, 2008, 2014
# Adaptação e correção de bugs por Hendrik Macedo, 2017

import numpy as np
import time

def makeTSP(nCities):
	positions = 2*np.random.rand(nCities,2)-1;
	distances = np.zeros((nCities,nCities))

	for i in range(nCities):
		for j in range(i+1,nCities):
			distances[i,j] = np.sqrt((positions[i,0] - positions[j,0])**2 + (positions[i,1] - positions[j,1])**2);
			distances[j,i] = distances[i,j];

	return distances

def exhaustive(distances):
	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],0]

	for newOrder in permutation(range(nCities)):
		possibleDistanceTravelled = 0
		for i in range(nCities-1):
			possibleDistanceTravelled += distances[newOrder[i],newOrder[i+1]]
		possibleDistanceTravelled += distances[newOrder[nCities-1],0]
			 
		if possibleDistanceTravelled < distanceTravelled:
			distanceTravelled = possibleDistanceTravelled
			cityOrder = newOrder

	return cityOrder, distanceTravelled
	
def permutation(order):
	order = tuple(order)
	if len(order)==1:
		yield order
	else:
		for i in range(len(order)):
			rest = order[:i] + order[i+1:]
			move = (order[i],)
			for smaller in permutation(rest):
				yield move + smaller
		
def greedy(distances):
	nCities = np.shape(distances)[0]
	distanceTravelled = 0
	
	# Need a version of the matrix we can trash
	dist = distances.copy()

	cityOrder = np.zeros(nCities, dtype=int)
	cityOrder[0] = np.random.randint(nCities)
	dist[:,int(cityOrder[0])] = np.Inf

	for i in range(nCities-1):
		cityOrder[i+1] = np.argmin(dist[cityOrder[i],:])
		distanceTravelled  += dist[cityOrder[i],cityOrder[i+1]]
		# Now exclude the chance of travelling to that city again
		dist[:,cityOrder[i+1]] = np.Inf
	
	# Now return to the original city
	distanceTravelled += distances[cityOrder[nCities-1],0]

	return cityOrder, distanceTravelled

def hillClimbing(distances):

	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)
	np.random.shuffle(cityOrder)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],0]

	for i in range(1000):
		# Choose cities to swap
		city1 = np.random.randint(nCities)
		city2 = np.random.randint(nCities)

		if city1 != city2:
			# Reorder the set of cities
			possibleCityOrder = cityOrder.copy()
			possibleCityOrder = np.where(possibleCityOrder==city1,-1,possibleCityOrder)
			possibleCityOrder = np.where(possibleCityOrder==city2,city1,possibleCityOrder)
			possibleCityOrder = np.where(possibleCityOrder==-1,city2,possibleCityOrder)

			# Work out the new distances
			# This can be done more efficiently
			newDistanceTravelled = 0
			for j in range(nCities-1):
				newDistanceTravelled += distances[possibleCityOrder[j],possibleCityOrder[j+1]]
			distanceTravelled += distances[cityOrder[nCities-1],0]
	
			if newDistanceTravelled < distanceTravelled:
				distanceTravelled = newDistanceTravelled
				cityOrder = possibleCityOrder

	return cityOrder, distanceTravelled
	

def simulatedAnnealing(distances):

	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)
	np.random.shuffle(cityOrder)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],0]

	T = 500
	c = 0.8
	nTests = 1000

	#while T>1:
	for i in range(nTests):
			# Choose cities to swap
			city1 = np.random.randint(nCities)
			city2 = np.random.randint(nCities)

			if city1 != city2:
				# Reorder the set of cities
				possibleCityOrder = cityOrder.copy()
				possibleCityOrder = np.where(possibleCityOrder==city1,-1,possibleCityOrder)
				possibleCityOrder = np.where(possibleCityOrder==city2,city1,possibleCityOrder)
				possibleCityOrder = np.where(possibleCityOrder==-1,city2,possibleCityOrder)

				# Work out the new distances
				# This can be done more efficiently
				newDistanceTravelled = 0
				for j in range(nCities-1):
					newDistanceTravelled += distances[possibleCityOrder[j],possibleCityOrder[j+1]]
				distanceTravelled += distances[cityOrder[nCities-1],0]

				if (newDistanceTravelled < distanceTravelled) or ((distanceTravelled - newDistanceTravelled) > T*np.log(np.random.rand())):
					distanceTravelled = newDistanceTravelled
					cityOrder = possibleCityOrder

			# Annealing schedule
			T = c*T

	return cityOrder, distanceTravelled

def parameters(nCities):
#	import time
#	nCities = ncities
	distances = makeTSP(nCities)
	return distances
    
def runExaustive(distances):
	print ("Busca exaustiva")
	start = time.time()
	result = exhaustive(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

def runGreedy(distances):
	print ("\nGreedy")
	start = time.time()
	result = greedy(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

def runHill(distances):
	print ("\nHill Climbing")
	start = time.time()
	result = hillClimbing(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

def runSimAnnealing(distances):
	print ("\nSimulated Annealing")
	start = time.time()
	result = simulatedAnnealing(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

#runAll()

In [2]:
distances = parameters(10)

In [3]:
runGreedy(distances)


Greedy
Ordem: [2 4 0 1 8 3 5 7 9 6]  Distancia: 5.063013761323302
Tempo: 0.00022602081298828125


In [4]:
runHill(distances)


Hill Climbing
Ordem: [5 6 9 7 4 2 1 0 8 3]  Distancia: 6.50705726882019
Tempo: 0.04918622970581055


In [5]:
runSimAnnealing(distances)


Simulated Annealing
Ordem: [1 0 2 4 9 5 7 6 8 3]  Distancia: 6.474767251545515
Tempo: 0.051300764083862305


In [6]:
runExaustive(distances)

Busca exaustiva
Ordem: (2, 6, 9, 7, 5, 3, 8, 1, 4, 0)  Distancia: 4.455394180099353
Tempo: 36.366687297821045
